In [1]:
%load_ext autoreload
%autoreload 2

# `Logit` on Orders - Logistic Regression (~1h)

## Select features

🎯 Haydi `wait_time` ve `delay_vs_expected` değişkenlerinin çok `iyi/kötü review`lar üzerindeki etkisini inceleyelim.

👉 `orders` training_set’imizi kullanarak iki adet `multivariate logistic regression` çalıştıracağız:
- `logit_one` → `dim_is_one_star` tahmini için  
- `logit_five` → `dim_is_five_star` tahmini için.

 

In [2]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

👉 Dataset’inizi import edin:

In [3]:
import sys
for path in sys.path:
    print(path)

/home/funny/sprint-15/01-Statistical-Inference/data-context-and-setup
/home/funny/data-logit
/home/funny/.pyenv/versions/3.12.9/lib/python312.zip
/home/funny/.pyenv/versions/3.12.9/lib/python3.12
/home/funny/.pyenv/versions/3.12.9/lib/python3.12/lib-dynload

/home/funny/.pyenv/versions/3.12.9/envs/workintech/lib/python3.12/site-packages


In [6]:
import olist
print(olist.__file__)

/home/funny/sprint-15/01-Statistical-Inference/data-context-and-setup/olist/__init__.py


In [4]:
from olist.order import Order
orders = Order().get_training_data(with_distance_seller_customer=True)

👉 Kullanmak istediğiniz feature’ları bir listede seçin:

⚠️ Data leakage yaratmadığınızdan emin olun (yani target’tan türetilmiş feature’ları seçmeyin)

💡 `wait_time` ve `delay_vs_expected` değişkenlerinin etkisini anlayabilmek için diğer feature’ların etkisini kontrol etmemiz gerekir, bu yüzden listenize ilgili olabilecek tüm feature’ları dahil edin.

In [7]:
print(orders.columns)

Index(['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected',
       'order_status', 'dim_is_five_star', 'dim_is_one_star', 'review_score',
       'number_of_sellers', 'price', 'freight_value', 'number_of_items',
       'distance_seller_customer'],
      dtype='object')


In [8]:
selected_features = [
    "wait_time",
    "delay_vs_expected",
    "number_of_sellers",
    "distance_seller_customer",
    "price",
    "freight_value"
    ]

🕵🏻 Feature’larınızın `multicollinearity` durumunu `VIF index` kullanarak kontrol edin.

* Çok yüksek olmamalıdır (tercihen < 10), böylece partial regression coefficient’larına ve ilgili `p-values` değerlerine güvenebiliriz.
* Verinizi standardize etmeyi unutmayın!
    * Bir `VIF Analysis`, bir feature’ın diğer feature’lara karşı regresyonunu yaparak hesaplanır...
    * Bu yüzden herhangi bir linear regression çalıştırmadan önce feature’ların `scale etkisini kaldırmak` ve eşit öneme sahip olmalarını sağlamak istersiniz!
    
    
📚 <a href="https://www.statisticshowto.com/variance-inflation-factor/">Statistics How To - Variance Inflation Factor</a>

📚  <a href="https://online.stat.psu.edu/stat462/node/180/">PennState - Detecting Multicollinearity Using Variance Inflation Factors</a>

⚖️ Standardize etme:

In [9]:
orders_standardized=orders.copy()

mu = orders[selected_features].mean()
sigma = orders[selected_features].std()
orders_standardized[selected_features] = (orders[selected_features] - mu) / sigma
orders_standardized


,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_sellers,price,freight_value,number_of_items,distance_seller_customer
0,e481f51cbdc54678b7cc49136f2d6af7,-0.426476,15.544063,-0.159108,delivered,0,0,4,-0.162651,-0.478836,-0.557892,1,-0.981662
1,53cdb2fc8bc7dce0b6741e2150273451,0.144964,19.137766,-0.159108,delivered,0,0,4,-0.162651,-0.129714,-0.133422,1,0.435320
2,47770eb9100c2d0c44946d9cf07ec65d,-0.324102,26.639711,-0.159108,delivered,1,0,5,-0.162651,0.032431,-0.240446,1,-0.138136
3,949d5b44dbf5de918fe9c16f97b45f8a,0.083679,26.188819,-0.159108,delivered,1,0,5,-0.162651,-0.419763,0.000812,1,2.086752
4,ad21c59c0840e6cb83a9ceb5573f8159,-1.021139,12.112049,-0.159108,delivered,1,0,5,-0.162651,-0.518545,-0.557892,1,-0.962777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109464,63943bddc261676b46f01ca7ac2f7bd8,1.044190,23.459051,-0.159108,delivered,0,0,4,-0.162651,0.091464,-0.213841,1,-0.206679
109465,83c1379a015df1e13d02aae0204711ab,1.329158,30.384225,-0.159108,delivered,1,0,5,-0.162651,0.213820,1.144221,1,0.633262
109466,11c177c8e97725db2631073c19f07b62,0.498210,37.105243,-0.159108,delivered,0,0,2,-0.162651,0.819854,1.632784,2,-0.383123
109467,11c177c8e97725db2631073c19f07b62,0.498210,37.105243,-0.159108,delivered,0,0,2,-0.162651,0.819854,1.632784,2,-0.383123


👉 Olası multicollinearity durumlarını analiz etmek için VIF Analysis’inizi çalıştırın:

In [35]:
X_standardized = orders_standardized[selected_features]
X_standardized.head()


,wait_time,delay_vs_expected,number_of_sellers,distance_seller_customer,price,freight_value
0,-0.426476,-0.159108,-0.162651,-0.981662,-0.478836,-0.557892
1,0.144964,-0.159108,-0.162651,0.435320,-0.129714,-0.133422
2,-0.324102,-0.159108,-0.162651,-0.138136,0.032431,-0.240446
3,0.083679,-0.159108,-0.162651,2.086752,-0.419763,0.000812
4,-1.021139,-0.159108,-0.162651,-0.962777,-0.518545,-0.557892


In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
vif_df=pd.DataFrame()
vif_df["features"] = X_standardized.columns
vif_df["vif_index"] = [vif(X_standardized.values, i) for i in range(X_standardized.shape[1])]
round(vif_df,2)

,features,vif_index
0,wait_time,2.58
1,delay_vs_expected,2.17
2,number_of_sellers,1.03
3,distance_seller_customer,1.36
4,price,1.20
5,freight_value,1.29


## Logistic Regressions

👉 İki adet `Logistic Regression` modeli fit edin:
- `logit_one` → `dim_is_one_star` tahmini için
- `logit_five` → `dim_is_five_star` tahmini için.

`Logit 1️⃣`

In [39]:
logit_one = smf.logit(formula='dim_is_one_star ~'+ ' + '.join(selected_features), data=orders_standardized).fit()
logit_one.summary()

Optimization terminated successfully.
         Current function value: 0.310463
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:        dim_is_one_star   No. Observations:               109461
Model:                          Logit   Df Residuals:                   109454
Method:                           MLE   Df Model:                            6
Date:                Mon, 29 Dec 2025   Pseudo R-squ.:                  0.1267
Time:                        22:26:41   Log-Likelihood:                -33984.
converged:                       True   LL-Null:                       -38913.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.2440      0.011   -204.871      0.000      -2.265      -2.223
wait_time                    0.6058      0.015     40.887      0.000       0.577       0.635
delay_vs_expected            0.2769      0.017     16.608      0.000       0.244       0.310
number_of_sellers            0.3029      0.007     41.989      0.000       0.289       0.317
distance_seller_customer    -0.2135      0.012    -18.006      0.000      -0.237      -0.190
price                        0.0701      0.010      7.158      0.000       0.051       0.089
freight_value                0.1985      0.010     19.373      0.000       0.178       0.219
============================================================================================
"""

`Logit 5️⃣`

In [40]:
logit_five = smf.logit(formula='dim_is_five_star ~'+ ' + '.join(selected_features), data=orders_standardized).fit()
logit_five.summary()

Optimization terminated successfully.
         Current function value: 0.642446
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:       dim_is_five_star   No. Observations:               109461
Model:                          Logit   Df Residuals:                   109454
Method:                           MLE   Df Model:                            6
Date:                Mon, 29 Dec 2025   Pseudo R-squ.:                 0.05761
Time:                        22:28:45   Log-Likelihood:                -70323.
converged:                       True   LL-Null:                       -74622.
Covariance Type:            nonrobust   LLR p-value:                     0.000
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    0.2676      0.007     40.302      0.000       0.255       0.281
wait_time                   -0.4866      0.011    -45.001      0.000      -0.508      -0.465
delay_vs_expected           -0.4212      0.022    -19.453      0.000      -0.464      -0.379
number_of_sellers           -0.2348      0.007    -31.758      0.000      -0.249      -0.220
distance_seller_customer     0.1159      0.008     15.323      0.000       0.101       0.131
price                        0.0076      0.007      1.076      0.282      -0.006       0.022
freight_value               -0.1599      0.009    -17.629      0.000      -0.178      -0.142
============================================================================================
"""

💡 Şimdi bu iki logistic regression’ın sonuçlarını analiz etme zamanı:

- Partial coefficient’ları kendi kelimelerinizle yorumlayın.
- `p-values` kullanarak istatistiksel anlamlılıklarını kontrol edin.
- Coefficient önemleri açısından `logit_one` ve `logit_five` arasında herhangi bir fark görüyor musunuz?

In [41]:
# Aşağıdaki cümlelerden doğru olanları aşağıdaki listeye kaydedin.

a = "delay_vs_expected influences five_star ratings even more than one_star ratings"
b = "wait_time influences five_star ratings even more than one_star"

your_answer = [a]

🧪 __Kodunu Test Et__

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult('logit',
    answers = your_answer
)
result.write()
print(result.check())

<details>
    <summary>- <i>Açıklamalar ve ileri seviye kavramlar</i> -</summary>


> _Diğer tüm şeyler sabitken, `delay factor`, 1-yıldız review alma ihtimalini etkilemesinden bile daha fazla, 5-yıldızdan mahrum kalma ihtimalini artırma eğilimindedir. Muhtemelen bunun sebebi, 1-yıldız review’ların bizzat çok kötü ürünleri hedeflemesi, kötü teslimatları değil._

❗️ Ancak tamamen titiz olmak için, **iki farklı modelin coefficient’larını karşılaştırırken daha dikkatli olmamız gerekir**, çünkü **benzer popülasyonlara dayanmayabilirler**!
    Burada 2 alt popülasyonumuz var: (1-yıldız verenler ve 5-yıldız verenler) ve bunlar doğaları gereği farklı davranış kalıpları sergileyebilirler. 5-yıldız vermeye daha meyilli “mutlu insanlar”ın, “gecikme” veya “fiyat” söz konusu olduğunda, 1-yıldızı “Lucky-Luke gibi ateşleyen” “huysuz insanlara” göre daha az hassas olmaları gayet mümkün...

</details>



🏁 Tebrikler!

💾 `logit.ipynb` notebook’unuzu commit ve push etmeyi unutmayın!